In [1]:
import numpy as np

In [2]:
from scipy.linalg import lu, solve

In [3]:
A = np.array([[2, 3, 1],
              [4, 7, 7],
              [6, 18, 22]], dtype=float)
b = np.array([1, 2, 3], dtype=float)

In [5]:
def fwd(L, b):
  n = L.shape[0]
  y = np.zeros_like(b)
  for i in range(n):
    y[i] = b[i] - L[i, :i] @ y[:i]

  return y

In [6]:
def bwd(U, y):
  n = U.shape[0]
  x = np.zeros_like(y)
  for i in reversed(range(n)):
    x[i] = (y[i] - U[i, i+1:] @ x[i+1:]) / U[i, i]
  return x

In [4]:
def lu(A):
  n = A.shape[0]
  P = np.eye(n)
  L = np.zeros((n, n))
  U = A.copy().astype(float)

  for k in range(n):
    max_index = np.argmax(abs(U[k:, k])) + k
    if k != max_index:
      U[[k, max_index]] = U[[max_index, k]]
      P[[k, max_index]] = P[[max_index, k]]
      L[[k, max_index]] = L[[max_index, k]]

    for i in range(k+1, n):
      L[i][k] = U[i][k] / U[k][k]
      U[i][k:] = U[i][k:] - L[i][k] * U[k][k:]

    np.fill_diagonal(L, 1)
    return P, L, U

In [7]:
P, L, U = lu(A)
Pb = P @ b
y = fwd(L, Pb)
x_custom = bwd(U, y)

P_scipy, L_scipy, U_scipy = lu(A)
x_scipy = solve(A, b)

print("Custom: ", x_custom)
print("Scipy: ", x_scipy)

Custom:  [ 0.5 -0.  -0. ]
Scipy:  [ 0.5 -0.  -0. ]


In [8]:
import time

In [10]:
import time

# Generate a random square matrix
n = 100  # Size of the matrix
A_random = np.random.rand(n, n)
b_random = np.random.rand(n)

# Benchmark your LU decomposition
start = time.time()
P_custom, L_custom, U_custom = lu(A_random)
Pb_random = P_custom @ b_random
y_random = fwd(L_custom, Pb_random)
x_custom_random = bwd(U_custom, y_random)
end = time.time()
custom_time = end - start
print(f"Custom LU time for size {n}: {custom_time:.6f} seconds")

# Benchmark SciPy LU decomposition
start = time.time()
P_scipy_random, L_scipy_random, U_scipy_random = lu(A_random)
x_scipy_random = solve(A_random, b_random)
end = time.time()
scipy_time = end - start
print(f"SciPy LU time for size {n}: {scipy_time:.6f} seconds")


Custom LU time for size 100: 0.002754 seconds
SciPy LU time for size 100: 0.004535 seconds
